# CHAPTER10　高可用性構成

## 10-1　ロードバランサー

・ロードバランサー（負荷分散装置）：サーバーにかかる負荷を、平等に振り分けるための装置<br>

AWSが提供するロードバランサーのサービスは、以下の通りである。

・ELB（Elastic Load Balancer）：CLB/ALB/NLBの総称<br>
・CLB（Classic Load Balancer）：レイヤー4機能とレイヤー7の一部機能を提供するロードバランサー<br>
・ALB（Application Load Balancer）：**レイヤー7**に特化したロードバランサー<br>
・NLB（Network Load Balancer）：**レイヤー4**に特化したロードバランサー<br>

ELBの特徴は、以下の通りである。

・フルマネージドな負荷分散装置<br>
・ヘルスチェック<br>
・高可用性（複数のアベイラビリティーゾーンに自動で配置される）<br>
・スケーラブル（負荷に応じてキャパシティが自動で増減する）<br>
・定番構成（1つのELB配下にEC2とRDSをアベイラビリティーゾーンに分散配置）がある<br>
・リバースプロキシとして利用することもできる<br>
・**自動でスケールイン・アウトする**<br>
・**グルーバルIPアドレスが動的に変化する**<br>
・**Internet Facing**（インターネット向けのELB）<br>
・**Internal**（内部向けのELB、インターネットからアクセスされない）<br>
・**SSL Termination**（ELB側でHTTPを受信し、バックエンドのサーバーはHTTP通信を受け取る構成を組み、SSL処理をELBにオフロードさせる機能）<br>
・**スティッキーセッション**（セッションを維持するために、一定期間同じEC2にリクエストを送信する機能）<br>

※オフロード：代わりに負荷を軽減させること<br>
※セッション：以前ログインしたことのあるユーザを識別する機能<br>
※SSL（Secure Sockets Layer）：送受信しているデータを暗号化する仕組み（URLにHTTTPSを付けたり、URLの横に鍵のマークを表示）

CLBの特徴は、以下の通りである。

・**HTTP/HTTPS/TCP/SSL**の負荷分散を行う<br>
・SSL Termination（SSL処理の負荷をオフロードする）

ALBの特徴は、以下の通りである。

・**HTTP/HTTPS**の負荷分散を行う<br>
・SSL Termination（SSL処理の負荷をオフロードする）<br>
・コンテンツベースルーティング（**ホストベースルーティング**、**パスベースルーティング**）

NLBの特徴は、以下の通りである。

・**TCPのみ**の負荷分散を行う<br>
・**SSL Termination機能はない**<br>
・グローバルIPアドレスを固定化する

## 10-2　オートスケール

・オートスケール：需要に応じてEC2インスタンスを自動的に拡張・縮小する機能

スケジューリングプラン（オートスケールの判断基準）には、以下の2つがある。

・動的スケジューリング：CloudWatchのメトリクスに基づいて、負荷に応じてインスタンスの追加・削除を行う<br>
・スケジュールベーススケーリング：事前に組んだスケジュールに沿って、インスタンスの追加・削除を行う

※メトリクス：データを数値化し、分析や計算などの加工を行ったもの<br>
※CloudWatch：AWSの各種リソースを監視・モニタリングする機能（メトリクスを保存してグラフ化を行う）

動的スケーリングには、以下の3種類がある。

・シンプルスケーリングポリシー：平均CPU使用率が80%で2台追加する<br>
・ステップスケーリングポリシー：平均CPU使用率が60%以上で2台（or 20%の台数）を追加、80%以上でもう2台（or 20%の台数）を追加する<br>
・ターゲット追跡スケーリングポリシー：平均CPU使用率を50%に維持するために必要な台数を自動集計し、指定したメトリクスを維持する

ヘルスチェック（インスタンスが正常かどうかをチェックする機能）には、以下の2種類がある。

・EC2ヘルスチェック：EC2インスタンスのステータスがrunning以外の場合に、異常と判定する<br>
・ELBヘルスチェック：ELBのヘルスチェックを利用する

オートスケールはEC2起動時にAMIベースで起動され、その際のアプリケーションの配置方法には以下の2つがある。

・ゴールデンイメージ方式：アプリケーションが**組み込まれている**、起動が速い、アプリケーションを変更する際にAMIを作り直す必要がある<br>
・ユーザーデータ方式：アプリケーションが**組み込まれていない**、変更に手間がない、アプリケーションを配備する際に時間がかかる

オートスケールでは、以下の2つが実行される。

・スケールイン：インスタンスの台数を**減らす**<br>
・スケールアウト：インスタンスの台数を**増やす**

スケールインを行うと、EC2インスタンスは削除されるため、以下の用途ではEC2インスタンスを利用するのは推奨されていない。

・データベースサーバー<br>
・ファイルサーバー

## 10-3　オートスケーリングによる耐障害性設計

EC2を複数のアベイラビリティーゾーンに分散することで、片方のアベイラビリティーゾーンが停止した場合でも<br>
もう片方で指定した台数が維持され、サービスが停止することなく一定量のEC2が稼働し続ける。

この構成の場合、以下の2つに応じてオートスケールさせることも可能である。

・平均CPU使用率<br>
・ELBのリクエスト数

## 10-4　オートスケールとインスタンス購入オプションによるコスト最適化

インスタンスの購入方法には、以下の種類がある。

・オンデマンドインスタンス：通常の購入方法<br>
・リザーブドインスタンス：事前に最低限使用する台数を予約する購入方法、最大75%引きで利用できる<br>
・スポットインスタンス：AWS上で余っている余剰リソースを購入する方法、最大90%引きで利用できる、入札（購入）金額を上回ると停止することがある

コスト最適化には、以下の2つの方法がある。

・最低台数をリザーブドインスタンス、追加台数をオンデマンドインスタンス<br>
・最低台数をオンデマンドインスタンス、追加台数をスポットインスタンス

## 10-5　Queueによる負荷分散と動的スケーリング

EC2のオートスケールは、SQSのメッセージ数（SendもしくはReceiveメトリクス）に応じて行われることがある。<br>
これはCloudwatchのメトリクスに、「SQSのメッセージ数」も含まれるからである。

・SQS（Amazon Simple Queue Service）：サーバーレスでキューイングをサポートするもの<br>
・キューイング（Queue）：送信側が受信側の状態に左右されずにメッセージを投げて、自分の仕事に戻ることができる仕組み

## 10-6　ElasitCache

・ElasitCache：AWSで低レイテンシーかつキャッシュを活用するための代表的なサービス<br>
・キャッシュ（cache）：アクセスしたサイトのデータや履歴を保管しておくこと<br>
・レイテンシー：データ送信における遅延時間

**RDSの前段にElasitCacheを配置**することで、**クエリ結果をキャッシュ**し、データベースへのアクセスや負荷を減らすことができる。

## 10-7　データベースの高可用性

・Multi-AZ：マスターデータベースとは別のAVZでスレーブデータベースを起動し、データベースを常に同期し、異常時にフェイルオーバーできる機能<br>
・可用性：システムが継続して稼働できる度合いや能力のこと<br>
・フェイルオーバー：コンピュータサーバ・システム・ネットワークで異常事態が発生したときに、待機システムに切り替えること